### Notebook 2 — Limpieza y preparación de datos Listado Partes

In [13]:
import numpy as np
import pandas as pd

pd.options.mode.copy_on_write = True # CoW por defecto a partir de Pandas 3.0.0

# configuración de visualización

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

In [14]:
# carga de data set

df_original_partes = pd.read_csv(
    "./datasets/ListadoPartes.csv",
    low_memory=False
)
#El dataset se cargó utilizando low_memory=False 
#con el fin de garantizar una inferencia consistente de los tipos de datos, evitando errores derivados de columnas con valores heterogéneos.

In [15]:
df = df_original_partes.copy()

In [16]:
# Normalización nombres de columnas para eliminar espacios
# y evitar errores por inconsistencias en los encabezados.

df.columns = df.columns.str.strip().str.lower()

In [17]:
print("Dataset cargado correctamente")
print("Dimensiones:", df.shape)

Dataset cargado correctamente
Dimensiones: (631, 26)


In [18]:
# verificacion inicial

print("\n--- TIPOS DE DATOS ---")
print(df.dtypes)

print("\n--- VALORES NULOS (%) ---")
print((df.isnull().mean() * 100).round(2))

print("\n--- DUPLICADOS ---")
print(df.duplicated().sum())


--- TIPOS DE DATOS ---
nif                       object
nombre                    object
apellido1                 object
apellido2                 object
fechaparteaccidente       object
fechaaltaparte            object
codigo                     int64
descripcionparte          object
actividad                 object
lugar                     object
fecharecepcion            object
centrourgencias           object
helicoptero              float64
macizomontañoso           object
nombreparaje              object
entrenamiento               bool
actividadpersonal           bool
actividadorganizada         bool
festivo                     bool
nresponsables              int64
tamañogrupo                int64
centrohospitalizacion     object
descripciongrado          object
pais                      object
provincia                 object
tipoaccidente             object
dtype: object

--- VALORES NULOS (%) ---
nif                        0.00
nombre                     0.00
apellido1    

In [19]:
#Eliminación de columnas 100% nulas

nulos = df.isna().mean()
cols_100_nulas = nulos[nulos == 1].index

df.drop(columns=cols_100_nulas, inplace=True)

In [20]:
print("\n--- VALORES NULOS (%) ---")
print((df.isnull().mean() * 100).round(2))


--- VALORES NULOS (%) ---
nif                       0.00
nombre                    0.00
apellido1                 0.00
apellido2                 0.16
fechaparteaccidente       0.00
fechaaltaparte            0.00
codigo                    0.00
descripcionparte          0.00
actividad                 0.00
lugar                     0.00
fecharecepcion            0.00
centrourgencias          59.27
macizomontañoso           0.48
nombreparaje              0.48
entrenamiento             0.00
actividadpersonal         0.00
actividadorganizada       0.00
festivo                   0.00
nresponsables             0.00
tamañogrupo               0.00
centrohospitalizacion    91.28
descripciongrado          6.66
pais                      0.00
provincia                 0.00
tipoaccidente             0.00
dtype: float64


In [23]:
# conversion de fechas 
# Conversión de variables de fecha a formato datetime
# para facilitar análisis temporal y detección de errores.

columnas_fecha = [
    'fechaparteaccidente',
    'fechaaltaparte',
    'fecharecepcion',
]

for col in columnas_fecha:
    df[col] = pd.to_datetime(df[col], errors='coerce')

In [24]:
# Numéricos
# Se garantiza la coherencia de las variables numéricas,
# convirtiendo valores inválidos en NaN.

columnas_numericas = df.select_dtypes(include=["int64", "float64"]).columns

for col in columnas_numericas:
    df[col] = pd.to_numeric(df[col], errors="coerce")

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 631 entries, 0 to 630
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   nif                    631 non-null    object        
 1   nombre                 631 non-null    object        
 2   apellido1              631 non-null    object        
 3   apellido2              630 non-null    object        
 4   fechaparteaccidente    631 non-null    datetime64[ns]
 5   fechaaltaparte         631 non-null    datetime64[ns]
 6   codigo                 631 non-null    int64         
 7   descripcionparte       631 non-null    object        
 8   actividad              631 non-null    object        
 9   lugar                  631 non-null    object        
 10  fecharecepcion         631 non-null    datetime64[ns]
 11  centrourgencias        257 non-null    object        
 12  macizomontañoso        628 non-null    object        
 13  nombr

In [26]:
# Limpieza básica de texto para eliminar espacios innecesarios
# sin alterar el significado de las categorías.


columnas_texto = df.select_dtypes(include=["object"]).columns

for col in columnas_texto:
    df[col] = (
        df[col]
        .astype(str)
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
    )

### Normalización de variables

En esta fase se realiza una normalización básica de las variables categóricas.
Primero se corrigen errores estructurales de texto (espacios y formato) y,
posteriormente, se normalizan semánticamente las variables clave implicadas
en el análisis de siniestralidad para garantizar la consistencia de las categorías.


In [27]:
# Normalización estructural de variables de texto
# Se corrigen espacios innecesarios y problemas de formato
# manteniendo los valores nulos reales (NaN).

columnas_texto = df.select_dtypes(include="object").columns

for col in columnas_texto:
    df[col] = (
        df[col]
        .str.strip()
        .str.replace(r"\s+", " ", regex=True)
    )


In [28]:
# Normalización semántica (mayúsculas) en variables categóricas relevantes
# para análisis de frecuencias y comparación de perfiles de siniestro.

columnas_categoricas_clave = [
    "descripciongrado",
    "tipoaccidente",
    "actividad",
    "lugar",
    "pais",
    "provincia"
]

for col in columnas_categoricas_clave:
    if col in df.columns:
        df[col] = df[col].str.upper()


In [29]:
# Comprobación tras normalización
df[columnas_categoricas_clave].nunique().sort_values()


descripciongrado      4
pais                 13
actividad            20
provincia            36
tipoaccidente        42
lugar               467
dtype: int64

### Análisis de cardinalidad de variables categóricas

Se analiza la cardinalidad de las variables categóricas del dataset de
siniestros una vez finalizada la limpieza y normalización, con el objetivo
de evaluar su viabilidad para el análisis exploratorio y comparativo.
Este análisis permite identificar variables con un número elevado de
categorías que pueden requerir agrupación o exclusión en fases posteriores
del análisis.


In [30]:
# Análisis de cardinalidad de variables categóricas
cardinalidad = (
    df.select_dtypes(include="object")
      .nunique()
      .sort_values()
)

cardinalidad


descripciongrado           4
pais                      13
actividad                 20
provincia                 36
centrohospitalizacion     38
tipoaccidente             42
centrourgencias          206
nombre                   278
macizomontañoso          318
apellido1                366
apellido2                386
nombreparaje             458
lugar                    467
nif                      589
descripcionparte         627
dtype: int64

### Interpretación de la cardinalidad

El análisis de cardinalidad del dataset de siniestros muestra una adecuada
estructura en las variables clave relacionadas con la gravedad del accidente,
la actividad y la localización general. Por el contrario, se identifican
variables con alta cardinalidad asociadas a texto libre, localización específica
o datos identificativos, las cuales no se consideran adecuadas para el análisis
exploratorio ni para la contrastación de hipótesis, aunque se mantienen en el
dataset limpio por motivos de trazabilidad.


In [33]:
# Guardado del dataset de partes limpio
# Este dataset se utilizará en el análisis exploratorio conjunto
# y en la contrastación de hipótesis.

ruta_partes_limpio = "./datasets/siniestros_limpio.csv"

df.to_csv(
    ruta_partes_limpio,
    index=False,
    encoding="utf-8-sig"
)


Una vez finalizada la limpieza, normalización y análisis de cardinalidad
del dataset de siniestros, se guarda el conjunto de datos limpio para su
uso en el análisis exploratorio conjunto y la contrastación de hipótesis
en notebooks posteriores.

